In [2]:
import sys
sys.path.append("C:/Users/rohit/OneDrive/Documents/Rohit Michigan/AIPLUS")
from Gesture_Music_Control.GestureClass import Handlandmarks
from Gesture_Music_Control.extra_functions import extra_functions
import cv2
import os
import numpy as np

### Data Collection

In [3]:
folder_name = "my_data"
actions = ['gesture 1', 'gesture 2','gesture 3','gesture 4','gesture 5','gesture 6','gesture 7']
frames = 30
videos = 40

In [9]:
folder_setup = extra_functions(no_sequence=videos, sequence_length=frames, data_path=folder_name, actions=actions)
# callling the make folders function'
folder_setup.make_folders()

In [14]:
landmarks = None
cap = cv2.VideoCapture(0)
timestamp = 0
for action in actions:
    for sequence in range(videos):
        for frame_num in range(frames):
            keypoints=[]
            ret, frame = cap.read()
            handlandmarks = Handlandmarks(keypoints, frame, timestamp)
            handlandmarks.get_landmarks()
            handconnections = handlandmarks.returnhandmarks()
            landmarks = handlandmarks.returngesture()
            frame = handlandmarks.markings(frame, handconnections)
            
            if frame_num == 0 :
                cv2.putText(frame, ' starting collection', ( 120, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 4, cv2.LINE_AA)
                cv2.putText(frame, ' collecting frames for {} video number {}'.format(action, sequence), ( 15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)
                cv2.imshow('frame', frame)
                cv2.waitKey(1000)
            else:
                cv2.putText(frame, ' collecting frames for {} video number {}'.format(action, sequence), ( 15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)
                cv2.imshow('frame', frame)
            
            #print(landmarks)
            npy_path = os.path.join(folder_name, action, str(sequence), str(frame_num))
            #print(npy_path)
            np.save(npy_path, landmarks)

            timestamp = timestamp + 1
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
cap.release()
cv2.destroyAllWindows()

In [15]:
cap.release()
cv2.destroyAllWindows()

### Train LSTM

import notes:
Train without any play option and increase the number of videos.

In [1]:
import sys
sys.path.append("C:/Users/rohit/OneDrive/Documents/Rohit Michigan/AIPLUS")
from Gesture_Music_Control.GestureClass import Handlandmarks
from Gesture_Music_Control.extra_functions import extra_functions
import cv2
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [10]:
folder_setup = extra_functions(no_sequence=videos, sequence_length=frames, data_path=folder_name, actions=actions)

In [11]:
folder_setup.creating_label_map()

In [12]:
label_map = folder_setup.return_label_map()

In [13]:
label_map

{'gesture 1': 0,
 'gesture 2': 1,
 'gesture 3': 2,
 'gesture 4': 3,
 'gesture 5': 4,
 'gesture 6': 5,
 'gesture 7': 6}

In [14]:
sequences, labels = folder_setup.concating_gesture_sequences(path=folder_name, no_sequence = videos, sequence_length = frames, actions = actions, label_map=label_map)

In [15]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)
print("shape of X Dataset:",X.shape)
print("shape of y Dataset:",y.shape)

shape of X Dataset: (280, 30, 63)
shape of y Dataset: (280, 7)


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
print("shape of X train Dataset:",X_train.shape)
print("shape of X test Dataset:",X_test.shape)
print("shape of y train Dataset:",y_train.shape)
print("shape of y test Dataset:",y_test.shape)

shape of X train Dataset: (196, 30, 63)
shape of X test Dataset: (84, 30, 63)
shape of y train Dataset: (196, 7)
shape of y test Dataset: (84, 7)


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

log_dir = os.path.join("logs")
tb_callback = TensorBoard(log_dir=log_dir)

model = Sequential()
model.add(LSTM(1024,return_sequences=True, activation='relu', input_shape=(30,63)))
model.add(LSTM(512,return_sequences=True, activation='relu',))
model.add(LSTM(512,return_sequences=True, activation='relu'))
model.add(LSTM(128,return_sequences=True, activation='relu'))
model.add(LSTM(128,return_sequences=True, activation='relu',dropout=0.1))
model.add(LSTM(128,return_sequences=False, activation='relu',dropout=0.1))
model.add(Dense(128, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(np.array(actions).shape[0], activation='softmax'))    

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 1024)          4456448   
                                                                 
 lstm_1 (LSTM)               (None, 30, 512)           3147776   
                                                                 
 lstm_2 (LSTM)               (None, 30, 512)           2099200   
                                                                 
 lstm_3 (LSTM)               (None, 30, 128)           328192    
                                                                 
 lstm_4 (LSTM)               (None, 30, 128)           131584    
                                                                 
 lstm_5 (LSTM)               (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 128)               1

In [4]:
## Orginal model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

log_dir = os.path.join("logs")
tb_callback = TensorBoard(log_dir=log_dir)

model = Sequential()
model.add(LSTM(512, return_sequences=True, activation='relu', input_shape=(30,63)))
model.add(LSTM(64, return_sequences=False, activation='relu', dropout=0.1))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(np.array(actions).shape[0], activation='softmax'))    

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 512)           1179648   
                                                                 
 lstm_1 (LSTM)               (None, 64)                147712    
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 7)                 231       
                                                                 
Total params: 1,333,831
Trainable params: 1,333,831
Non-trainable params: 0
_________________________________________________________________


In [37]:
model.fit(X_train, y_train, epochs=500, callbacks=[tb_callback]) # you only need 300 epochs

Epoch 1/500
7/7 [==============================] - 3s 166ms/step - loss: 1.9363 - categorical_accuracy: 0.1990
Epoch 2/500
7/7 [==============================] - 1s 156ms/step - loss: 1.8199 - categorical_accuracy: 0.5153
Epoch 3/500
7/7 [==============================] - 1s 155ms/step - loss: 1.6041 - categorical_accuracy: 0.4337
Epoch 4/500
7/7 [==============================] - 1s 162ms/step - loss: 1.6331 - categorical_accuracy: 0.6531
Epoch 5/500
7/7 [==============================] - 1s 157ms/step - loss: 1.6104 - categorical_accuracy: 0.5153
Epoch 6/500
7/7 [==============================] - 1s 158ms/step - loss: 1.5427 - categorical_accuracy: 0.4439
Epoch 7/500
7/7 [==============================] - 1s 168ms/step - loss: 1.3038 - categorical_accuracy: 0.6837
Epoch 8/500
7/7 [==============================] - 1s 182ms/step - loss: 1.1146 - categorical_accuracy: 0.7551
Epoch 9/500
7/7 [==============================] - 1s 166ms/step - loss: 2.3345 - categorical_accuracy: 0.7143
E

In [38]:
model.save('MyFinalModel.h5')

Eval Matrix

In [39]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
y_hat = model.predict(X_test)
y_true = np.argmax(y_test, axis=1).tolist()
y_hat = np.argmax(y_hat, axis=1).tolist()
multilabel_confusion_matrix(y_true, y_hat)

3/3 [==============================] - 0s 52ms/step


array([[[70,  0],
        [ 0, 14]],

       [[74,  0],
        [ 0, 10]],

       [[72,  0],
        [ 0, 12]],

       [[74,  0],
        [ 0, 10]],

       [[73,  0],
        [ 0, 11]],

       [[72,  0],
        [ 0, 12]],

       [[69,  0],
        [ 0, 15]]], dtype=int64)

In [40]:
accuracy_score(y_true, y_hat)

1.0

Real Time Prediction

In [19]:
import sys
sys.path.append("C:/Users/rohit/OneDrive/Documents/Rohit Michigan/AIPLUS")
from Gesture_Music_Control.GestureClass import Handlandmarks
from Gesture_Music_Control.extra_functions import extra_functions
from Gesture_Music_Control.AudioControls import AudioControlsClass
import cv2
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import pyautogui

In [20]:
model.load_weights("MyFinalModel.h5")

In [21]:
"""p = AudioControlsClass()
sequence = []
threshold = 0.4
landmarks = None
predictions = []
gesture = 'Nothing'
timestamp=0
cap = cv2.VideoCapture(0)
while True:
    keypoints=[]
    ret, frame = cap.read()
    handlandmarks = Handlandmarks(keypoints, frame, timestamp)
    handlandmarks.get_landmarks()
    landmarks = handlandmarks.returngesture()
    landmarks = np.array(landmarks).flatten()
    sequence.append(landmarks)
    sequence = sequence[-frames:]
    if len(sequence) == frames:
        res = model.predict(np.expand_dims(sequence, axis=0))[0]
        if res[np.argmax(res)]> threshold:
            predictions.append(actions[np.argmax(res)])
            if  len(predictions) > 10:
                gesture = p.prediction_gesture_handler(predictions)
                if gesture == "Pause":
                    p.pause_video_vlc()
                elif gesture == "Volumn Up":
                    p.take_screenshot()
                elif gesture == "Volumn Down":
                    p.decrease_volume()
    cv2.rectangle(frame, (0,0), (640,40),(245,117,16),-1)
    cv2.putText(frame, str(gesture), (3,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255),2,cv2.LINE_AA)
    handconnections = handlandmarks.returnhandmarks()
    frame = handlandmarks.markings(frame, handconnections)
    cv2.imshow('frame', frame)
    timestamp = timestamp + 1
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()"""

'p = AudioControlsClass()\nsequence = []\nthreshold = 0.4\nlandmarks = None\npredictions = []\ngesture = \'Nothing\'\ntimestamp=0\ncap = cv2.VideoCapture(0)\nwhile True:\n    keypoints=[]\n    ret, frame = cap.read()\n    handlandmarks = Handlandmarks(keypoints, frame, timestamp)\n    handlandmarks.get_landmarks()\n    landmarks = handlandmarks.returngesture()\n    landmarks = np.array(landmarks).flatten()\n    sequence.append(landmarks)\n    sequence = sequence[-frames:]\n    if len(sequence) == frames:\n        res = model.predict(np.expand_dims(sequence, axis=0))[0]\n        if res[np.argmax(res)]> threshold:\n            predictions.append(actions[np.argmax(res)])\n            if  len(predictions) > 10:\n                gesture = p.prediction_gesture_handler(predictions)\n                if gesture == "Pause":\n                    p.pause_video_vlc()\n                elif gesture == "Volumn Up":\n                    p.take_screenshot()\n                elif gesture == "Volumn Down"

In [26]:
import time
p = AudioControlsClass()
sequence = []
threshold = 0.4
landmarks = None
predictions = []
gesture = 'Nothing'
timestamp=0
cap = cv2.VideoCapture(0)
while True:
    keypoints=[]
    ret, frame = cap.read()
    handlandmarks = Handlandmarks(keypoints, frame, timestamp)
    handlandmarks.get_landmarks()
    landmarks = handlandmarks.returngesture()
    landmarks = np.array(landmarks).flatten()
    sequence.append(landmarks)
    sequence = sequence[-frames:]
    if len(sequence) == frames:
        res = model.predict(np.expand_dims(sequence, axis=0))[0]
        if res[np.argmax(res)]> threshold:
            predictions.append(actions[np.argmax(res)])
            #gesture = predictions[-1]
            gesture = p.prediction_gesture_handler(predictions)
            if gesture == "gesture 1":
                p.increase_volume()
                gesture = "Volumn Up"
            elif gesture == "gesture 2":
                p.decrease_volume()
                gesture = "Volumn Down"
            elif gesture == "gesture 3":
                p.switch_to_previous_desktop() 
                gesture = "Switching to previous Tab"               
            elif gesture == "gesture 4":
                p.switch_to_previous_desktop() 
                gesture = "Switching to next Tab"
            elif gesture == "gesture 5":
                p.take_screenshot()
                gesture = "Take Screenshot"
            elif gesture == "gesture 6":
                p.trigger_file_explorer()
                gesture = "open file explorer"
            

    cv2.putText(frame, str(gesture), (3,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255),2,cv2.LINE_AA)
    handconnections = handlandmarks.returnhandmarks()
    frame = handlandmarks.markings(frame, handconnections)
    cv2.imshow('frame', frame)
    timestamp = timestamp + 1
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 27ms/step


Exception ignored in: <function _compointer_base.__del__ at 0x000001DBC4CC5090>
Traceback (most recent call last):
  File "c:\Users\rohit\AppData\Local\Programs\Python\Python310\lib\site-packages\comtypes\__init__.py", line 614, in __del__
    self.Release()
  File "c:\Users\rohit\AppData\Local\Programs\Python\Python310\lib\site-packages\comtypes\__init__.py", line 847, in Release
    return self.__com_Release()
OSError: exception: access violation reading 0x0000000000000009


1/1 [==============================] - 0s 36ms/step


In [24]:
cap.release()
cv2.destroyAllWindows()